In [1]:
import pandas as pd
import numpy as np

In [2]:
# df = pd.read_csv('glassdoor_reviews.csv')

# df = df.drop(['date_review','current'], axis=1)

# text_cols = df.select_dtypes(include='object').columns
# for col in text_cols:
#     df[col] = df[col].fillna('')
    
# def define_level(rating):
#     if rating >= 4:
#         return 'high'
#     if rating <= 2:
#         return 'low'
#     return 'middle'

# df['rating_level'] = df['overall_rating'].apply(lambda x: define_level(x))

# df['combined_text'] = df[['headline', 'pros', 'cons']].apply(lambda x: ''.join(x), axis=1)

# def deal_empty(string):
#     if string.isspace():
#         return ""
#     else:
#         return string

# df['firm'] = df['firm'].apply(lambda x: deal_empty(x))
# df['job_title'] = df['job_title'].apply(lambda x: deal_empty(x))
# df['location'] = df['location'].apply(lambda x: deal_empty(x))

# df.to_csv('UI_data.csv')

In [2]:
df = pd.read_csv('UI_data.csv', index_col = 0)
df.head()

,firm,job_title,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline,pros,cons,rating_level,combined_text
0,AFH-Wealth-Management,NaN,NaN,2,4.0,3.0,NaN,2.0,3.0,3.0,x,o,r,"Young colleagues, poor micro management",Very friendly and welcoming to new staff. Easy...,"Poor salaries, poor training and communication.",low,"Young colleagues, poor micro managementVery fr..."
1,AFH-Wealth-Management,Office Administrator,"Bromsgrove, England, England",2,3.0,1.0,NaN,2.0,1.0,4.0,x,o,r,"Excellent staff, poor salary","Friendly, helpful and hard-working colleagues",Poor salary which doesn't improve much with pr...,low,"Excellent staff, poor salaryFriendly, helpful ..."
2,AFH-Wealth-Management,Office Administrator,"Bromsgrove, England, England",1,1.0,1.0,NaN,1.0,1.0,1.0,x,o,x,"Low salary, bad micromanagement",Easy to get the job even without experience in...,"Very low salary, poor working conditions, very...",low,"Low salary, bad micromanagementEasy to get the..."
3,AFH-Wealth-Management,NaN,NaN,5,2.0,3.0,NaN,2.0,2.0,3.0,x,o,r,Over promised under delivered,Nice staff to work with,No career progression and salary is poor,high,Over promised under deliveredNice staff to wor...
4,AFH-Wealth-Management,Office Administrator,"Bromsgrove, England, England",1,2.0,1.0,NaN,2.0,1.0,1.0,x,o,x,client reporting admin,"Easy to get the job, Nice colleagues.","Abysmal pay, around minimum wage. No actual tr...",low,"client reporting adminEasy to get the job, Nic..."


In [1]:
%%writefile myapp.py
import streamlit as st
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns

import spacy
from spacy.lang.en import English
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.parsing.preprocessing import STOPWORDS


nlp = spacy.load("en_core_web_sm")

### Data Import ###
df = pd.read_csv('UI_data.csv', index_col = 0)
color_dict = {'1. FC Köln': '#fc4744', '1. FC Nürnberg':'#8c0303', '1. FC Union Berlin':'#edd134', '1. FSV Mainz 05':'#fa2323', 'Bayer 04 Leverkusen':'#cf0c0c', 'Bayern München':'#e62222', 'Bor. Mönchengladbach':'#1f9900', 'Borussia Dortmund':'#fff830', 'Eintracht Braunschweig':'#dbca12', 'Eintracht Frankfurt':'#d10606', 'FC Augsburg':'#007512', 'FC Ingolstadt 04':'#b50300', 'FC Schalke 04':'#1c2afc', 'Fortuna Düsseldorf':'#eb3838', 'Hamburger SV':'#061fc2', 'Hannover 96':'#127a18', 'Hertha BSC':'#005ac2', 'RB Leipzig':'#0707a8', 'SC Freiburg':'#d1332e', 'SC Paderborn 07':'#0546b5', 'SV Darmstadt 98':'#265ade', 'TSG Hoffenheim':'#2b82d9', 'VfB Stuttgart':'#f57171', 'VfL Wolfsburg':'#38d433', 'Werder Bremen':'#10a30b'}
ratings = ['overall_rating','work_life_balance','culture_values','diversity_inclusion','career_opp','comp_benefits','senior_mgmt']

st.set_page_config(layout="wide")

def plot_rating(df): #df is filtered
    rc = {'figure.figsize':(6,3),
          'axes.facecolor':'white',
          'axes.edgecolor': 'white',
          'axes.labelcolor': 'grey',
          'figure.facecolor': 'white',
          'patch.edgecolor': 'white',
          'text.color': 'grey',
          'xtick.color': 'grey',
          'ytick.color': 'grey',
          'grid.color': 'grey',
          'font.size' : 8,
          'axes.labelsize': 12,
          'xtick.labelsize': 8,
          'ytick.labelsize': 12}
    
    plt.rcParams.update(rc)
    fig, ax = plt.subplots()
    
    if not sort_rating:
        df_plot = df[ratings].mean()    
    else:
        df_plot = df[ratings].mean().sort_values(ascending=False)
        
    if compare_analysis_rating:
        df_plot_2 = filtered_df_2[ratings].mean()
        
        # create DataFrame for df_plot and df_plot_2 separately
        df_plot = pd.DataFrame({'values': df_plot.values, 'group': 'original'}, index=df_plot.index)
        df_plot_2 = pd.DataFrame({'values': df_plot_2.values, 'group': 'new'}, index=df_plot_2.index)

        # concatenate vertically to create a new DataFrame
        compare_plot = pd.concat([df_plot, df_plot_2])

        # plot a double bar chart using seaborn
        sns.barplot(data=compare_plot, x=compare_plot.index, y='values', hue='group', palette=['#6dcd8d', '#7cb4d6'])
        ax.legend(loc = 'lower right')
    else:
        ax = sns.barplot(x=df_plot.index, y=df_plot.values, color = "#6dcd8d")
        
    ax.set(xlabel = "Rating Aspect", ylabel = "Rating Score")
    plt.xticks(rotation=66, horizontalalignment="right")
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., p.get_height()),
               ha = 'center',
               va = 'center', 
               xytext = (0, 18),
               textcoords = 'offset points')
    st.pyplot(fig)
    
    
def extract_sentiment_measures(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    sentiment_subjectivity = blob.sentiment.subjectivity

    return pd.Series({
        'sentiment_polarity': sentiment_polarity,
        'sentiment_subjectivity': sentiment_subjectivity,
    })

def plot_sentiment(df): #df is filtered
    rc = {'figure.figsize':(6,3),
          'axes.facecolor':'white',
          'axes.edgecolor': 'white',
          'axes.labelcolor': 'grey',
          'figure.facecolor': 'white',
          'patch.edgecolor': 'white',
          'text.color': 'grey',
          'xtick.color': 'grey',
          'ytick.color': 'grey',
          'grid.color': 'grey',
          'font.size' : 8,
          'axes.labelsize': 12,
          'xtick.labelsize': 8,
          'ytick.labelsize': 12}
    
    plt.rcParams.update(rc)
    fig, ax = plt.subplots()
    
    # apply the function to each row of the original DataFrame to create a new DataFrame
    sentiment_df = df.iloc[:100,:]['combined_text'].apply(extract_sentiment_measures)
    df_plot = sentiment_df.mean()    
    ax = sns.barplot(x=df_plot.index, y=df_plot.values, color = "#6dcd8d")
    ax.set(xlabel = "Sentiment Aspect", ylabel = "Sentiment Score")
    for p in ax.patches:
        ax.annotate(format(p.get_height(), '.2f'), 
              (p.get_x() + p.get_width() / 2., p.get_height()),
               ha = 'center',
               va = 'center', 
               xytext = (0, 18),
               textcoords = 'offset points')
    st.pyplot(fig)

def plot_top_phrases(df): #df is filtered
    rc = {'figure.figsize':(6,3),
          'axes.facecolor':'white',
          'axes.edgecolor': 'white',
          'axes.labelcolor': 'grey',
          'figure.facecolor': 'white',
          'patch.edgecolor': 'white',
          'text.color': 'grey',
          'xtick.color': 'grey',
          'ytick.color': 'grey',
          'grid.color': 'grey',
          'font.size' : 8,
          'axes.labelsize': 12,
          'xtick.labelsize': 8,
          'ytick.labelsize': 12}
    
    plt.rcParams.update(rc)
    fig, ax = plt.subplots()
    
    if scope == "all":
        processed_list = filtered_df["combined_text"].apply(lambda x: nlp(x.lower()))
    else:
        processed_list = filtered_df[scope].apply(lambda x: nlp(x.lower()))
          
    # Extract noun phrases
    noun_phrases = []
    for doc in nlp.pipe(processed_list):
        for chunk in doc.noun_chunks:
            if chunk.text not in STOPWORDS:
                noun_phrases.append(chunk.text)

    # Count the frequency of each noun phrase
    df_plot = pd.Series(noun_phrases).value_counts().head(n)    
    ax = sns.barplot(x=df_plot.index, y=df_plot.values, color = "#6dcd8d")
    ax.set_title("Top {} frequent noun phrases in {}".format(n,scope))
    ax.set(xlabel = "Noun Phrase", ylabel = "Frequency")
    plt.xticks(rotation=66, horizontalalignment="right")
    for p in ax.patches:
        ax.annotate(format(int(p.get_height()), 'd'), 
              (p.get_x() + p.get_width() / 2., p.get_height()),
               ha = 'center',
               va = 'center', 
               xytext = (0, 18),
               textcoords = 'offset points')
    st.pyplot(fig)

def plot_wordcloud(df): #df is filtered
    rc = {'figure.figsize':(6,3),
          'axes.facecolor':'white',
          'axes.edgecolor': 'white',
          'axes.labelcolor': 'grey',
          'figure.facecolor': 'white',
          'patch.edgecolor': 'white',
          'text.color': 'grey',
          'xtick.color': 'grey',
          'ytick.color': 'grey',
          'grid.color': 'grey',
          'font.size' : 8,
          'axes.labelsize': 12,
          'xtick.labelsize': 8,
          'ytick.labelsize': 12}
    
    plt.rcParams.update(rc)
    fig, ax = plt.subplots()
    
    if scope_2 == "all":
        text = ' '.join(filtered_df["combined_text"]).lower()
    else:
        text = ' '.join(filtered_df[scope_2]).lower()

    # Generate a WordCloud object with the text and set the parameters
    wordcloud = WordCloud(stopwords=STOPWORDS, width=800, height=400, background_color='white', max_words=n_2, colormap='Greens').generate(text)
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    st.pyplot(fig)


####################
### INTRODUCTION ###
####################
row0_spacer1, row0_1, row0_spacer2, row0_2, row0_spacer3 = st.columns((.1, 2.3, .1, 1.3, .1))
with row0_1:
    st.title('Glassdoor Job Reviews - Analyzer')
with row0_2:
    st.text("")
    st.subheader('Streamlit App by [Lexi Lu](https://www.linkedin.com/in/zilinlu/)')
row3_spacer1, row3_1, row3_spacer2 = st.columns((.1, 3.2, .1))
with row3_1:
    st.markdown("This large dataset based on Glassdoor contains job descriptions and rankings among various criteria such as work-life balance, income, culture, etc. The data covers the various industry.")
    st.markdown("You can find the database available in [Kaggle](https://www.kaggle.com/datasets/davidgauthier/glassdoor-job-reviews)")
    st.markdown("You can find the source code in the [GitHub Repository](https://github.com/)")
    
#################
### SELECTION ###
#################
filtered_df = df.copy()

st.sidebar.text('')
st.sidebar.text('')
st.sidebar.text('')
st.sidebar.markdown("**Make your selections of interest here:** 👇")
### COMPANY ###
firm_options = sorted([x for x in filtered_df['firm'].unique() if x != ''])
firm = st.sidebar.selectbox('First select the company you want to analyze:', firm_options, key = "option1_firm")
filtered_df = filtered_df[filtered_df['firm'] == firm]  

### JOB TITLE ###
job_options = sorted([str(x) for x in filtered_df['job_title'].unique() if x != ''])
job_title = st.sidebar.multiselect('Then select one or multiple job titles (optional):', job_options, key = "option1_job")
if job_title:
    filtered_df = filtered_df[filtered_df['job_title'].isin(job_title)]     

### LOCATION ###
location_options = sorted([str(x) for x in filtered_df['location'].unique() if x != ''])
location = st.sidebar.multiselect('Then select one or multiple locations (optional):', location_options, key = "option1_loc")
if location:
    filtered_df = filtered_df[filtered_df['location'].isin(location)]
    
### SEE DATA ###
row6_spacer1, row6_1, row6_spacer2 = st.columns((.2, 7.1, .2))
with row6_1:
    st.subheader("Currently selected data:")

row2_spacer1, row2_1, row2_spacer2, row2_2, row2_spacer3, row2_3, row2_spacer4, row2_4, row2_spacer5   = st.columns((.2, 1.6, .2, 1.6, .2, 1.6, .2, 1.6, .2))
with row2_1:
    str_reviews = "❤️ " + str(filtered_df.shape[0]) + " Reviews"
    st.markdown(str_reviews)
with row2_2:
    str_firms = "🧡 " + str(len(filtered_df['firm'].value_counts())) + " Companies"
    st.markdown(str_firms)
with row2_3:
    str_jobs = "💛 " + str(len(filtered_df['job_title'].value_counts())) + " Job Titles"
    st.markdown(str_jobs)
with row2_4:
    str_locations = "💚 " + str(len(filtered_df['location'].value_counts())) + " Locations"
    st.markdown(str_locations)

row3_spacer1, row3_1, row3_spacer2 = st.columns((.2, 7.1, .2))
with row3_1:
    st.markdown("")
    if filtered_df.shape[0] == 0:
        st.warning('There is no result for your current selection.')
    else:
        see_data = st.expander('You can click here to see the raw data first 👉')
        with see_data:
            st.dataframe(data=filtered_df.iloc[:,:-2].sort_values(by=['firm','job_title']).reset_index(drop=True))
st.text('')

################
### ANALYSIS ###
################

### RATINGS ###
row4_spacer1, row4_1, row4_spacer2 = st.columns((.2, 7.1, .2))
with row4_1:
    st.subheader('Average Rating Analysis')
row5_spacer1, row5_1, row5_spacer2, row5_2, row5_spacer3  = st.columns((.2, 2.3, .4, 4.4, .2))
with row5_1:
    st.text('')
    st.markdown('The graph on the right shows the average rating of your selected reviews data across different aspects including Career Opportunities, Comp & Benefits, Culture & Values, Senior Management, and Work/Life Balance', unsafe_allow_html=True, )   
    sort_rating = st.checkbox("Sort these ratings from high to low")
    st.text('')
    compare_analysis_rating = st.checkbox("Compare this with another option", key = "compare_rating")
    if compare_analysis_rating:
        filtered_df_2 = df.copy()
        ### COMPANY ###
        firm_options = sorted([x for x in filtered_df_2['firm'].unique() if x != ''])
        firm = st.selectbox('First select the company you want to compare:', firm_options, key = "option2_firm")
        filtered_df_2 = filtered_df_2[filtered_df_2['firm'] == firm]  
        ### JOB TITLE ###
        job_options = sorted([str(x) for x in filtered_df_2['job_title'].unique() if x != ''])
        job_title = st.multiselect('Then select one or multiple job titles (optional):', job_options, key = "option2_job")
        if job_title:
            filtered_df_2 = filtered_df_2[filtered_df_2['job_title'].isin(job_title)]     
        ### LOCATION ###
        location_options = sorted([str(x) for x in filtered_df_2['location'].unique() if x != ''])
        location = st.multiselect('Then select one or multiple locations (optional):', location_options, key = "option2_loc")
        if location:
            filtered_df_2 = filtered_df_2[filtered_df_2['location'].isin(location)]
with row5_2:
    plot_rating(filtered_df)

### SENTIMENT ANALYSIS ###
row6_spacer1, row6_1, row6_spacer2 = st.columns((.2, 7.1, .2))
with row6_1:
    st.subheader('Average Sentiment Analysis')
row7_spacer1, row7_1, row7_spacer2, row7_2, row7_spacer3  = st.columns((.2, 2.3, .4, 4.4, .2))
with row7_1:
    st.text('')
    st.markdown('sentiment_polarity: This is a measure of the sentiment polarity of the text, ranging from -1 (most negative) to 1 (most positive). A value of 0 indicates a neutral sentiment.', unsafe_allow_html=True, ) 
    st.markdown('sentiment_subjectivity: This is a measure of the subjectivity of the text, ranging from 0 (most objective) to 1 (most subjective). A value of 0 indicates an objective text, while a value of 1 indicates a highly subjective text.', unsafe_allow_html=True, )    
    st.text('')
    # compare_analysis = st.checkbox("Compare this with another option", key = "compare_sentiment")
with row7_2:
    plot_sentiment(filtered_df)

### PHRASE FREQUENCY ANALYSIS ###
row8_spacer1, row8_1, row8_spacer2 = st.columns((.2, 7.1, .2))
with row8_1:
    st.subheader('Top Frequent Noun Phrases Analysis')
row9_spacer1, row9_1, row9_spacer2, row9_2, row9_spacer3  = st.columns((.2, 2.3, .4, 4.4, .2))
with row9_1:
    st.text('')
    st.markdown('Show top frequent noun phrases')    
    scope = st.selectbox ("Which piece of text do you want to analyze?", ['all','pros','cons','headline'], key='unique_key_1')
    n = st.select_slider ("How many top phrases do you want to see?", options=[5, 10, 15, 20, 25, 30], value=15)
with row9_2:
    plot_top_phrases(filtered_df)

### WORDCLOUD ANALYSIS ###
row10_spacer1, row10_1, row10_spacer2 = st.columns((.2, 7.1, .2))
with row10_1:
    st.subheader('Wordcloud Analysis')
row11_spacer1, row11_1, row11_spacer2, row11_2, row11_spacer3  = st.columns((.2, 2.3, .4, 4.4, .2))
with row11_1:
    st.text('')   
    scope_2 = st.selectbox ("Which piece of text do you want to analyze?", ['all','pros','cons','headline'], key='unique_key_2')
    n_2 = st.select_slider ("What is the maximum number of words you want to see?", options=[50, 100, 150, 200], value=100)
with row11_2:
    plot_wordcloud(filtered_df)

################
### PREDICT ####
################
row12_spacer1, row12_1, row12_spacer2 = st.columns((.2, 7.1, .2))
with row12_1:
    st.subheader('Overall Rating Prediction')
with st.form(key="my_form"):
    row13_spacer1, row13_1, row13_spacer2, row13_2, row13_spacer3, row13_3, row13_spacer4 = st.columns((.2, 2, .2, 2, .2, 2, .2))
    with row13_1:
        text_headline = st.text_area(
        # Instructions
        "Please enter headline here",
        # 'sample' variable that contains our keyphrases.
        "Good firm!",
        # The height
        height=200,
        key="1")
    with row13_2:
        text_pros = st.text_area(
        # Instructions
        "Please enter pros here",
        # 'sample' variable that contains our keyphrases.
        "nice environment, friendly people",
        # The height
        height=200,
        key="2")
    with row13_3:
        text_cons = st.text_area(
        # Instructions
        "Please enter cons here",
        # 'sample' variable that contains our keyphrases.
        "no particular cons",
        # The height
        height=200,
        key="3")
        
    submit_button = st.form_submit_button(label="Submit")
    
### CONDITIONAL STATEMENTS ###
if submit_button and text_headline == '' and text_pros == '' and text_cons == '':
     st.warning("❄️ Please make sure to input something for rating prediction")
elif submit_button:
    from joblib import load
    from sklearn.feature_extraction.text import TfidfVectorizer
    import scipy
    from scipy.sparse import hstack

    test_df = pd.DataFrame({
    'headline': [text_headline],
    'pros': [text_pros],
    'cons': [text_cons]
    })
    
    # Load the saved model from a file
    logreg = load('C:/Users/lenovo/Documents/KaggleX/logreg_model.joblib')

    # Use the loaded model to make predictions
    test_features_lst = []
    for i in ['headline', 'pros', 'cons']:
        tfidf_vectorizer = load('C:/Users/lenovo/Documents/KaggleX/'+i+'_vectorizer.joblib')
        test_features = tfidf_vectorizer.transform(test_df[i].values)
        test_features_lst.append(test_features)

    X_test = hstack(test_features_lst)
    y_pred = logreg.predict(X_test)
    st.success("✅ Done! " + "The rating of this company is predicted to be " + y_pred)

Overwriting myapp.py


In [ ]:
!streamlit run myapp.py